# CiTO Citation Nanopublication Generator

This notebook generates typed citation nanopublications using the CiTO (Citation Typing Ontology) from a JSON configuration file.

## CiTO Citation Types
CiTO provides semantic annotation of citation intent, including:
- `cito:cites` - General citation
- `cito:usesDataFrom` - Uses data from cited work
- `cito:usesMethodIn` - Uses method from cited work
- `cito:extends` - Extends work from cited paper
- `cito:confirms` - Confirms findings of cited work
- `cito:obtainsBackgroundFrom` - Gets background information from cited work

In [ ]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_uri, make_literal, validate_required_fields,
    PREFIXES
)

In [ ]:
class CitoNanopubGenerator(NanopubGenerator):
    """Generator for CiTO citation nanopublications."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': config.get('template_uri'),
            'label': nanopub_config.get('nanopub_label', 'CiTO Citation')
        }
        super().__init__(merged_config)
        self.add_prefix('cito')
        self.add_prefix('fabio')
    
    def generate_assertion(self) -> str:
        """Generate the CiTO assertion graph."""
        citing_article = self.config['citing_article']
        citations = self.config.get('citations', [])
        
        # Normalize citing article URI
        if not citing_article.startswith('http'):
            citing_article = f"https://doi.org/{citing_article}"
        
        lines = [f"{self.sub_prefix}:assertion {{"]
        
        # Declare the citing article as a scholarly work
        lines.append(f"  <{citing_article}> a fabio:ScholarlyWork .")
        
        # Add each citation with its type
        for citation in citations:
            cited_article = citation['cited_article']
            citation_type = citation['citation_type']
            label = citation.get('label', '')
            
            # Normalize cited article URI
            if not cited_article.startswith('http'):
                cited_article = f"https://doi.org/{cited_article}"
            
            # Add the citation relationship
            lines.append(f"  <{citing_article}> {citation_type} <{cited_article}> .")
            
            # Add label for the cited work if provided
            if label:
                lines.append(f"  <{cited_article}> rdfs:label {make_literal(label)} .")
        
        lines.append("}")
        return "\n".join(lines)

In [ ]:
# Configuration
CONFIG_FILE = "../config/vbae208_cito.json"  # Change this to use different config
OUTPUT_DIR = "../output/cito"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"DOI: {config['metadata']['source_paper']['doi']}")
print(f"Number of CiTO nanopublications to generate: {len(config['nanopublications'])}")
print()

print("Available CiTO types:")
for cito_type, label in config.get('cito_types', {}).items():
    print(f"  {cito_type}: {label}")
print()

print("Citations to generate:")
for i, np_config in enumerate(config['nanopublications'], 1):
    for citation in np_config.get('citations', []):
        print(f"{i}. {citation['citation_type']} -> {citation.get('label', citation['cited_article'])}")

In [ ]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = CitoNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

In [ ]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

## Next Steps

1. Review the generated `.trig` files in the output directory
2. Sign and publish using Nanodash or nanopub-java
3. To use with a different paper, create a new JSON config file and update `CONFIG_FILE`